In [ ]:
%%javascript
IPython.notebook.clear_all_output();

In [ ]:
%reset -f
from IPython import get_ipython
get_ipython().magic('reset -sf')

%who

In [1]:
import os
import glob
import h5py
import shutil
import imgaug as aug
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.image as mimg
import imgaug.augmenters as iaa
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
import cv2
# from keras import backend as K
color = sns.color_palette()
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D
from tensorflow.keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
os.environ['AUTOGRAPH_VERBOSITY'] = "10"
tf.autograph.set_verbosity(0)
tf.compat.v1.logging.set_verbosity(0)

import sys
import datetime
import time

print("Python version: ", sys.version)
print("Version info.: ", sys.version_info)
print("TensorFlow version: ", tf.__version__)
print("TensorFlow.Keras version : ", tf.keras.__version__)

Python version:  3.6.10 |Anaconda, Inc.| (default, Jan  7 2020, 21:14:29) 
[GCC 7.3.0]
Version info.:  sys.version_info(major=3, minor=6, micro=10, releaselevel='final', serial=0)
TensorFlow version:  2.1.0
TensorFlow.Keras version :  2.2.4-tf


In [ ]:

# Turn interactive plotting off
# plt.ioff()
# Set the seed for hash based operations in python
os.environ['PYTHONHASHSEED'] = '0'

# Set the numpy seed
np.random.seed(111)

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
config.gpu_options.visible_device_list='0,1,2,3'
# config.gpu_options.visible_device_list='0,1'

# Set the random seed in tensorflow at graph level
tf.compat.v1.set_random_seed(111)

# Set the session in tensorflow
sess = tf.compat.v1.Session(config=config)

# Set the session in keras
tf.compat.v1.keras.backend.set_session(sess)

# tf.debugging.set_log_device_placement(True)
strategy = tf.distribute.MirroredStrategy()
# strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

print("#--#--"*10)
print('Number of devices: {}\n'.format(strategy.num_replicas_in_sync))

# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

# Make the augmentation sequence deterministic
aug.seed(111)

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
FileTime = str(datetime.datetime.now().strftime("%m-%d-%Y-%H-%M"))
print("#--#--"*10, "\nFile time: ", FileTime, "\n\n")

# Define path to the data directory
data_dir = Path('/data/user/tr27p/Courses/CS765-DeepLearning/FinalProject/Chest_X-Ray_Images_Pneumonia/Data/new_chest_xray/')

# Path to train directory (Fancy pathlib...no more os.path!!)
train_dir = data_dir / 'train'

# Path to validation directory
val_dir = data_dir / 'val'

# Path to test directory
test_dir = data_dir / 'test'


In [ ]:
with strategy.scope():
    train_list_ds = tf.data.Dataset.list_files(str(train_dir/"*"/"*"))
    test_list_ds = tf.data.Dataset.list_files(str(test_dir/"*"/"*"))
    val_list_ds = tf.data.Dataset.list_files(str(val_dir/"*"/"*"))

In [ ]:
CLASS_NAMES = np.array(['NORMAL','PNEUMONIA'])
def get_label(file_path):
    label = None
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    print(parts)
    # The second to last is the class-directory
    return parts[-2] == CLASS_NAMES

def decode_img(img, IMG_WIDTH=299, IMG_HEIGHT=299):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [ ]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
        plt.axis('off')

In [ ]:
with strategy.scope():
    train_labeled_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
    test_labeled_ds = test_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
    val_labeled_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

# for image, label in labeled_ds.take(1):
#     print("Image shape: ", image.numpy().shape)
#     print("Label: ", label.numpy())


In [ ]:
# BATCH_SIZE = 32
# BUFFER_SIZE = 1000
IMG_SIZE = 299
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

nb_epochs = 1
BUFFER_SIZE = 16

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [ ]:
# with strategy.scope():
#     train_batches = train_labeled_ds.repeat().cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
#     validation_batches = val_labeled_ds.repeat().batch(BATCH_SIZE)
#     test_batches = test_labeled_ds.batch(BATCH_SIZE)

train_batches = train_labeled_ds.repeat().batch(BATCH_SIZE).shuffle(BUFFER_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
validation_batches = val_labeled_ds.repeat().batch(BATCH_SIZE)
test_batches = test_labeled_ds.batch(BATCH_SIZE)

In [ ]:
# image_batch, label_batch = next(iter(train_batches))
# show_batch(image_batch.numpy(), label_batch.numpy())

In [ ]:
# train_size = !ls $train_dir/*/*.jpeg | wc -l
# test_size = !ls $test_dir/*/*.jpeg | wc -l
# val_size = !ls $val_dir/*/*.jpeg | wc -l

normal_cases_dir = train_dir / 'NORMAL'
pneumonia_cases_dir = train_dir / 'PNEUMONIA'
train_size = len(list(normal_cases_dir.glob('*.jpeg'))) + len(list(pneumonia_cases_dir.glob('*.jpeg')))
normal_cases_dir = test_dir / 'NORMAL'
pneumonia_cases_dir = test_dir / 'PNEUMONIA'
test_size = len(list(normal_cases_dir.glob('*.jpeg'))) + len(list(pneumonia_cases_dir.glob('*.jpeg')))
normal_cases_dir = val_dir / 'NORMAL'
pneumonia_cases_dir = val_dir / 'PNEUMONIA'
val_size = len(list(normal_cases_dir.glob('*.jpeg'))) + len(list(pneumonia_cases_dir.glob('*.jpeg')))

# train_size, test_size, val_size = int(train_size[0]), int(test_size[0]), int(val_size[0])

print("#--#--"*10,"\ntraining: {},\nvalidation: {},\ntest: {}".format(train_size, 
                                                                        val_size, 
                                                                        test_size))

In [ ]:
def print_inventory(inventory_name, dct):
    print('%s :' %(inventory_name))
    for item, amount in dct.items():  # dct.iteritems() in Python 2
        print('%15s : %s' % (item, amount))
        
# Callback for printing the LR at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print('\nLearning rate for epoch {} is {}\n'.format(epoch + 1, model.optimizer.lr.numpy()))


In [ ]:
learning_rate = 0.0001
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = learning_rate
exp_decay = tf.compat.v1.train.exponential_decay(starter_learning_rate,
                                                 global_step, 100000, 
                                                 0.96, staircase=True)

callbacks = [
#     tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ReduceLROnPlateau(monitor='val_auc', factor=0.3, patience=5, verbose=1, mode='max', min_delta=0.0001),
    # ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max'),
    # EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, mode='min', restore_best_weights=True),

    # EarlyStopping(  monitor='val_loss', min_delta=1e-3, patience=5, verbose=1,
    #                 mode='auto', baseline=None, restore_best_weights=True),
    ModelCheckpoint(filepath='/data/user/tr27p/Courses/CS765-DeepLearning/FinalProject/Chest_X-Ray_Images_Pneumonia/Python/best_model_todate_python/'+FileTime+'/',
                    monitor='val_auc', verbose=1, mode='max',
                    save_best_only=True, save_weights_only=True),
#     tf.keras.callbacks.LearningRateScheduler(exp_decay),
    PrintLR()
]

In [ ]:
# Define the number of training steps
nb_train_steps = train_size//BATCH_SIZE

# Define the number of validation steps
nb_valid_steps = val_size//BATCH_SIZE

# Define the number of validation steps
nb_test_steps = test_size//BATCH_SIZE

print("#--#--"*10,"\n\nNumber of training and validation steps: {} and {}".format(nb_train_steps, 
                                                                                  nb_valid_steps))

print("#--#--"*10,  "\n BATCH_SIZE_PER_REPLICA = ", BATCH_SIZE_PER_REPLICA,
                    "\n BATCH_SIZE = ", BATCH_SIZE,
                    "\n EPOCHS = ", nb_epochs)

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
with strategy.scope():
    base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                                   include_top=False,
                                                   weights='imagenet')
    base_model.trainable = False

In [ ]:
base_model.name

In [ ]:
base_model.summary()

In [ ]:
with strategy.scope():
    inputs = base_model.output
    flat = Flatten(name='flatten')(inputs)
    dense_1 = Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), name='fc1')(flat)
    drop_1 = Dropout(0.7, name='dropout1')(dense_1)
    dense_2 = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), name='fc2')(drop_1)
    drop_2 = Dropout(0.5, name='dropout2')(dense_2)
    predict = Dense(2, activation='softmax', name='fc3')(drop_2)

    model = Model(inputs=base_model.input, outputs=predict)
    model._name = "FrozenModel"

    opt = tf.keras.optimizers.Adam(lr=learning_rate, amsgrad=True, clipnorm=1., decay=0.004)
#     opt = tf.keras.optimizers.Nadam()
#     model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy', tf.keras.metrics.AUC()])
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
    
model.summary()

In [ ]:
print("#--#--"*10)
print_inventory("Optimizer", opt.get_config())
# print("#--#--"*10)
# print_inventory("Early Stopping", opt.get_config())
# print("#--#--"*10)
# print_inventory("Reduce LR On Plateau", opt.get_config())

print('\nReduce LR On Plateau :\n%15s : %s,\n%15s : %s,\n%15s : %s,\n%15s : %s'%('Monitor', callbacks[0].monitor,
                                                                                 'Factor', callbacks[0].factor,
                                                                                 'Mode', callbacks[0].mode,
                                                                                 'Patience', callbacks[0].patience) )

# print('\n Early Stopping:\n%15s : %s,\n%15s : %s,\n%15s : %s' %('Monitor', callbacks[1].monitor,
#                                                                 'MinDelta', callbacks[1].min_delta,
#                                                                 'patience', callbacks[1].patience) )

In [ ]:
model.metrics_names

In [ ]:
with strategy.scope():
    history = model.fit(train_batches, epochs=nb_epochs, steps_per_epoch=nb_train_steps,
                        validation_data=validation_batches, validation_steps=nb_valid_steps,
                        callbacks=callbacks, verbose=1)#,[chkpt, PrintLR()],
                        # class_weight={0:1.0, 1:0.4})

In [ ]:
# modelPath = './xray-best-model/best_model/best_model_'+datetime.datetime.now().strftime("%m-%d-%Y-%I-%M")+'.h5'
weightPath = '/data/user/tr27p/Courses/CS765-DeepLearning/FinalProject/Chest_X-Ray_Images_Pneumonia/Python/xray-best-model/best_model/best_model_para-tune_'+FileTime+'.hdf5'
# model.save(modelPath)
model.save_weights(weightPath)
# print("modelPath: ", modelPath)
print("#--#--"*10,"\n\nweightPath: ", weightPath)

In [ ]:
with strategy.scope():
    base_model_reLoad = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                                          include_top=False,
                                                          weights='imagenet')
    for layer in base_model_reLoad.layers:
        base_model_reLoad.trainable = True

In [ ]:
base_model_reLoad.summary()

In [ ]:
with strategy.scope():
    inputs = base_model_reLoad.output

    x = Flatten(name='flatten')(inputs)
    x = Dense(1024, activation='relu', name='fc1')(x)
    x = Dropout(0.7, name='dropout1')(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    x = Dense(2, activation='softmax', name='fc3')(x)
    
    model_reLoad = Model(inputs=base_model_reLoad.input, outputs=x)
    model_reLoad._name = "UnfrozenModel"

    opt = tf.keras.optimizers.Adam(lr=learning_rate, amsgrad=True, clipnorm=1.)
#     model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy', tf.keras.metrics.AUC()])
    model_reLoad.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

In [ ]:
model_reLoad.summary()

In [ ]:
model_reLoad.metrics_names

In [ ]:
with strategy.scope():
    model_reLoad.load_weights(weightPath)

In [ ]:
# def run_new_model(return_dict):
with strategy.scope():
    history_reLoad = model_reLoad.fit(train_batches, epochs=nb_epochs, steps_per_epoch=nb_train_steps,
                                      validation_data=validation_batches, validation_steps=nb_valid_steps,
                                      callbacks=callbacks, verbose=1)#,[chkpt, PrintLR()],
                                    # class_weight={0:1.0, 1:0.4})
#     return_dict['history'] = history_reLoad
#     print(return_dict.values())
#     return return_dict

In [ ]:
# history_reLoad = run_new_model()
# import multiprocessing
# manager = multiprocessing.Manager()
# return_dict = manager.dict()
# process_eval = multiprocessing.Process(target=run_new_model, args=(return_dict,))
# process_eval.start()
# process_eval.join()

In [ ]:
# print(return_dict.values())

In [ ]:
print("#--#--"*10,"\n\nTraining Completed \n\n")
print("history items : ", history_reLoad.history.keys())

In [ ]:
h = history_reLoad
fig = plt.figure()
plt.plot(h.history['accuracy'])
plt.plot(h.history['val_accuracy'])
plt.plot(np.argmax(h.history["accuracy"]),
         np.max(h.history["val_accuracy"]),
         marker="x", color="b", label="best model")
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train-accuracy', 'val-accuracy'], loc='upper left')
#plt.savefig("/data/user/tr27p/Courses/CS765-DeepLearning/FinalProject/Chest_X-Ray_Images_Pneumonia/Python/python_acc.png", format='png')
# plt.savefig('/data/user/tr27p/Courses/CS765-DeepLearning/FinalProject/Chest_X-Ray_Images_Pneumonia/Python/plots/acc/python_acc-'+FileTime+'.png', format='png')
plt.show()
plt.close(fig)

In [ ]:
h = history_reLoad
fig = plt.figure()
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.plot(np.argmin(h.history["loss"]),
         np.min(h.history["val_loss"]),
         marker="x", color="r", label="best model")
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train-loss', 'val-loss'], loc='upper left')
# plt.savefig('/data/user/tr27p/Courses/CS765-DeepLearning/FinalProject/Chest_X-Ray_Images_Pneumonia/Python/plots/loss/python_loss-'+FileTime+'.png', format='png')
plt.show()
plt.close()

In [ ]:
h = history_reLoad
fig = plt.figure()
plt.plot(h.history['auc_1'])
plt.plot(h.history['val_auc_1'])
plt.plot(np.argmax(h.history["auc_1"]),
         np.max(h.history["val_auc_1"]),
         marker="x", color="b", label="best model")
plt.title('model AUC (Area under the curve)')
plt.ylabel('AUC')
plt.xlabel('epoch')
plt.legend(['train-AUC', 'val-AUC'], loc='upper left')
#plt.savefig("/data/user/tr27p/Courses/CS765-DeepLearning/FinalProject/Chest_X-Ray_Images_Pneumonia/Python/python_acc.png", format='png')
# plt.savefig('/data/user/tr27p/Courses/CS765-DeepLearning/FinalProject/Chest_X-Ray_Images_Pneumonia/Python/plots/auc/python_auc-'+FileTime+'.png', format='png')
plt.show()
plt.close(fig)

In [ ]:
# modelPath = './xray-best-model/best_model/best_model_'+datetime.datetime.now().strftime("%m-%d-%Y-%I-%M")+'.h5'
weightPath = '/data/user/tr27p/Courses/CS765-DeepLearning/FinalProject/Chest_X-Ray_Images_Pneumonia/Python/xray-best-model/best_model/best_model_para-tune_'+FileTime+'.hdf5'
# model.save(modelPath)
model_reLoad.save_weights(weightPath)
# print("modelPath: ", modelPath)
print("#--#--"*10,"\n\nweightPath: ", weightPath)

In [ ]:
with strategy.scope():
    model_reLoad.load_weights(weightPath)

In [ ]:
# Evaluation on test dataset
with strategy.scope():
    test_loss, test_score, test_auc = model_reLoad.evaluate(test_batches)
print("Loss on test set: ", test_loss)
print("Accuracy on test set: ", test_score)
print("AUC on test set: ", test_auc)


In [ ]:
pred_test_labels=[]
orig_test_labels=[]
for X_sample, Y_sample in test_batches.take(4):
    print(X_sample.shape, Y_sample.shape)
#     with strategy.scope():
    pred = model_reLoad.predict(X_sample)
    for pY, oY in zip(pred, Y_sample):
        pred_test_labels.append(pY)
        orig_test_labels.append(oY)
        
pred_test_labels = np.argmax(pred_test_labels, axis=-1)
orig_test_labels = np.argmax(orig_test_labels, axis=-1)

print("#--#--"*10,"\n\n",orig_test_labels.shape)
print(pred_test_labels.shape)

In [ ]:
# Get the confusion matrix
cm  = confusion_matrix(orig_test_labels, pred_test_labels)
fig = plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True,cmap=plt.cm.Blues)
plt.xticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.yticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
#plt.savefig("/data/user/tr27p/Courses/CS765-DeepLearning/FinalProject/Chest_X-Ray_Images_Pneumonia/Python/python_confusion-mat.png", format='png')
# plt.savefig('/data/user/tr27p/Courses/CS765-DeepLearning/FinalProject/Chest_X-Ray_Images_Pneumonia/Python/plots/confusion-mat/python_confusion-mat-'+FileTime+'.png', format='png')
plt.show()
plt.close()

# In[ ]:


# Calculate Precision and Recall
tn, fp, fn, tp = cm.ravel()

precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1_score = (2*precision*recall/(precision+recall))
print("#--#--"*10,"\n\nRecall of the model is {:.2f}".format(recall))
print("Precision of the model is {:.2f}".format(precision))
print("F1-score: {}".format(f1_score))

In [ ]:
# Define path to the data directory
tes_data_dir = Path('/data/user/tr27p/Courses/CS765-DeepLearning/FinalProject/Chest_X-Ray_Images_Pneumonia/Data/chest_xray/chest_xray/')

# Path to test directory
test_dir_2 = tes_data_dir / 'test'

In [ ]:
with strategy.scope():
    test_2_list_ds = tf.data.Dataset.list_files(str(test_dir_2/"*"/"*"))

In [ ]:
with strategy.scope():
    test_2_labeled_ds = test_2_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
test_2_batches = test_2_labeled_ds.batch(BATCH_SIZE)

In [ ]:
pred_test_labels=[]
orig_test_labels=[]
for X_sample, Y_sample in test_2_batches.take(4):
    print(X_sample.shape, Y_sample.shape)
#     with strategy.scope():
    pred = model_reLoad.predict(X_sample)
    for pY, oY in zip(pred, Y_sample):
        pred_test_labels.append(pY)
        orig_test_labels.append(oY)
        
pred_test_labels = np.argmax(pred_test_labels, axis=-1)
orig_test_labels = np.argmax(orig_test_labels, axis=-1)

print("#--#--"*10,"\n\n",orig_test_labels.shape)
print(pred_test_labels.shape)

In [ ]:
# Get the confusion matrix
cm  = confusion_matrix(orig_test_labels, pred_test_labels)
fig = plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True,cmap=plt.cm.Blues)
plt.xticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.yticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
#plt.savefig("/data/user/tr27p/Courses/CS765-DeepLearning/FinalProject/Chest_X-Ray_Images_Pneumonia/Python/python_confusion-mat.png", format='png')
# plt.savefig('/data/user/tr27p/Courses/CS765-DeepLearning/FinalProject/Chest_X-Ray_Images_Pneumonia/Python/plots/confusion-mat/python_confusion-mat-'+FileTime+'.png', format='png')
plt.show()
plt.close()

# In[ ]:


# Calculate Precision and Recall
tn, fp, fn, tp = cm.ravel()

precision = tp/(tp+fp)
recall = tp/(tp+fn)

print("#--#--"*10,"\n\nRecall of the model is {:.2f}".format(recall))
print("Precision of the model is {:.2f}".format(precision))
print("F1-score: {}".format(2*precision*recall/(precision+recall)))